In [76]:
# 1. 파이썬 코드에서 Matplotlib 폰트 설정
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import timedelta
from itertools import combinations
from collections import Counter

# 폰트 설정
plt.rc('font', family='Malgun Gothic')
# 마이너스 부호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

print("한글 폰트 설정이 완료되었습니다.")


한글 폰트 설정이 완료되었습니다.


In [77]:
# 3. 데이터 로드 및 통합 (모든 문제 풀이의 시작점)
# dataframe 확인해라!
try:
    orders_df = pd.read_csv('../data/orders.csv')
    payments_df = pd.read_csv('../data/payments.csv')
    products_df = pd.read_csv('../data/products.csv')
    shipping_df = pd.read_csv('../data/shipping.csv')
    customers_df = pd.read_csv('../data/customers.csv')

    # 모든 데이터프레임 병합
    df = pd.merge(orders_df, payments_df, on='order_id', how='left')
    df = pd.merge(df, products_df, on='product_id', how='left')
    df = pd.merge(df, customers_df, on='customer_id', how='left')
    df = pd.merge(df, shipping_df, on='order_id', how='left')

    # 데이터 전처리
    date_cols = ['order_date', 'payment_date', 'join_date', 'shipping_start_date', 'shipping_end_date']
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    df['total_sales'] = df['quantity'] * df['price']
    
    print("데이터 로드 및 통합이 완료되었습니다.")

except FileNotFoundError as e:
    print(f"파일을 찾을 수 없습니다: {e}")


데이터 로드 및 통합이 완료되었습니다.


In [ ]:
# 문제 10: 주문 취소가 가장 빈번한 상품 상위 5개를 찾고, 이들 상품의 공통적인 특징을 분석하세요.
# 비즈니스 목적: 취소율이 높은 상품의 원인을 파악(예: 불충분한 상품 설명, 높은 가격)하여 운영 효율성을 개선하고 매출 손실을 줄입니다.

In [ ]:
# 출력 결과를 보고 코딩하세요

주문 취소(결제 대기)가 잦은 상품 TOP 5:
product_name
화장품 상품_160     5
스포츠용품 상품_6     4
화장품 상품_256     4
전자제품 상품_159    4
의류 상품_132      4
Name: count, dtype: int64

해당 상품들의 특징:
             category  avg_price  avg_stock
product_name                               
스포츠용품 상품_6        화장품    74300.0        2.0
의류 상품_132          식품     6600.0       39.0
전자제품 상품_159        식품    57500.0       32.0
화장품 상품_160      스포츠용품    51600.0        5.0
화장품 상품_256         도서    16500.0       24.0


In [51]:
# 문제 10: 주문 취소가 가장 빈번한 상품 상위 5개를 찾고, 이들 상품의 공통적인 특징을 분석하세요.
df.columns


Index(['order_id', 'customer_id', 'product_id', 'order_date', 'quantity',
       'payment_id', 'payment_method', 'payment_status', 'payment_date',
       'product_name', 'category', 'price', 'stock', 'name', 'gender', 'age',
       'join_date', 'city', 'shipping_id', 'shipping_company',
       'shipping_status', 'shipping_start_date', 'shipping_end_date',
       'total_sales'],
      dtype='object')

In [ ]:
df[(df['product_name'] == '화장품 상품_256')][['product_id', 'product_name', 'category', 'payment_status', 'shipping_status']]

In [ ]:
# df['payment_status'].unique() # array(['Completed', 'Failed', 'Pending'], dtype=object)
# df['shipping_status'].unique() # array(['Delivered', 'Shipped', 'Preparing', nan, 'Cancelled'],  dtype=object)

# 주문 취소 상품 필요한 데이터 추출
need_cols = ['product_id', 'product_name', 'quantity', 'payment_status', 'shipping_status', 'category', 'price', 'stock']
df_quest10 = df[
    (df['payment_status'] == 'Pending') 
    # (df['shipping_status'].isna()) |
    # (df['shipping_status'] == 'Cancelled')
    ][need_cols].copy()
# df_quest10.info()
# df_quest10.head(20)

# TOP 5 출력 (product_name 인덱스의 Series)
print("주문 취소(결제 대기)가 잦은 상품 TOP 5:")
df_quest10['product_name'].value_counts().head(5)

# 전체 상품의 특징 구하기
print("\n해당 상품들의 특징:")
features = df_quest10.groupby('product_name').agg({
    'category': 'first',
    'price': 'mean',
    'stock': 'mean',
    'product_name': 'count'
}).rename(columns={'price': 'avg_price', 'stock': 'avg_stock', 'product_name': 'count'})

# count 기준으로 정렬 후 TOP 5만 출력 (count 컬럼 제외)
print(features.sort_values('count', ascending=False).head(5)[['category', 'avg_price', 'avg_stock']])

# 해당 상품들의 특징
print("\n해당 상품들의 특징:")
# count 기준으로 정렬 후 TOP 5만 출력 (count 컬럼 제외)
print(features.sort_values('count', ascending=False).head()[['category', 'avg_price', 'avg_stock']])

주문 취소(결제 대기)가 잦은 상품 TOP 5:

해당 상품들의 특징:
             category  avg_price  avg_stock
product_name                               
화장품 상품_160      스포츠용품    51600.0        5.0
의류 상품_132          식품     6600.0       39.0
전자제품 상품_159        식품    57500.0       32.0
화장품 상품_256         도서    16500.0       24.0
스포츠용품 상품_6        화장품    74300.0        2.0

해당 상품들의 특징:
             category  avg_price  avg_stock
product_name                               
화장품 상품_160      스포츠용품    51600.0        5.0
의류 상품_132          식품     6600.0       39.0
전자제품 상품_159        식품    57500.0       32.0
화장품 상품_256         도서    16500.0       24.0
스포츠용품 상품_6        화장품    74300.0        2.0
